# 三维重建第七课：相机标定代码讲解
https://www.bilibili.com/video/BV1sK4y1s7Qs/ 

In [1]:
# 导入python标准库
import numpy as np
import cv2
import glob #glob模块的主要方法就是glob，该方法返回所有匹配的文件路径列表(list)
#该方法需要一个参数用来指定匹配的路径字符串(字符串可以为绝对路径也可以为相对路径)
#其返回文件名只包括当前目录里的文件名，不包括子文件夹里的文件。

# numpy.zeros
numpy.zeros(shape,dtype=float,order='C'):返回给定形状和类型的全0数组。
# numpy.mgrid
numpy.mgrid 当其被索引时返回一个多维网格(如2D图形，3D图形)，输出矩阵的维度和数量等于索引维度的数量。

numpy.mgrid\[start:end:step\]：start是开始，end是结束坐标（实数不包括，复数就包括），step是步长
* 第1返回值是第1维数据在最终结构中的分布（分布以矩阵形式呈现）
* 第2返回值是第2维数据在最终结构中的分布
* 第3维(，第4维)以此类推

np.mgrid\[ \[1:3:3j, 4:5:2j\] \] 
* 3j : 一共3个点，包括结束坐标3
* 步长为复数表示点数，左闭右闭
* 步长为实数表示间隔，左闭右开

### np.mgrid举例：
#### 一维结构：
```python
>>> import numpy as np
>>> x = np.mgrid[-5:5:5j]
>>> x
array = ([-5.,-2.5, 0. , 2.5 , 5. ])
>>>
```
#### 二维结构：
```python
>>> import numpy as np
>>> x,y=np.mgrid[-5:5:3j,-2:2:3j]
>>> x
array([[-5., -5., -5.],
       [ 0.,  0.,  0.],
       [ 5.,  5.,  5.]])
>>> y
array([[-2.,  0.,  2.],
       [-2.,  0.,  2.],
       [-2.,  0.,  2.]])
>>> 
```
> 注意： 其中x沿着水平向右的方向扩展(即是：每列都相同)，观察x。y沿着垂直的向下的方向扩展(即是：每行都相同)。观察y。

如果步长是整数(则输出不包括结束坐标）：
```python
>>> import numpy as np
>>> x,y=np.mgrid[-5:5:3j,-2:2:3]
>>> x
array([[-5., -5.],
       [ 0.,  0.],
       [ 5.,  5.]])
>>> y
array([[-2.,  1.],
       [-2.,  1.],
       [-2.,  1.]])
>>> 
```
#### 三维结构：
```python
>>> import numpy as np
>>> x,y,z=np.mgrid[-5:5:3j,-2:2:3j,-1:1:2]
>>> x
array([[[-5.],
        [-5.],
        [-5.]],
 
       [[ 0.],
        [ 0.],
        [ 0.]],
 
       [[ 5.],
        [ 5.],
        [ 5.]]])
>>> y
array([[[-2.],
        [ 0.],
        [ 2.]],
 
       [[-2.],
        [ 0.],
        [ 2.]],
 
       [[-2.],
        [ 0.],
        [ 2.]]])
>>> z
array([[[-1.],
        [-1.],
        [-1.]],
 
       [[-1.],
        [-1.],
        [-1.]],
 
       [[-1.],
        [-1.],
        [-1.]]])
>>> 
```
三维绘图效果，见https://blog.csdn.net/abc13526222160/article/details/88559162

# numpy.ndarray.T
### examples:
#### 二维矩阵转置：
```python
>>> x = np.array([[1.,2.],[3.,4.]])
>>> x
array([[1.,2.],
      [3.,4.]])
>>> x.T
array([[1.,3.],
      [2.,4.]])
```
#### 一维向量转置：
```python
x = np.array([1.,2.,3.,4.])
x
array([ 1.,  2.,  3.,  4.])
x.T
array([ 1.,  2.,  3.,  4.])
```
# numpy.reshape
numpy.reshape(a,newshape,order='C')
* a 需要处理的数据
* newshape ：新格式，行数列数相乘后等于a中元素的数量。如果一个维度是-1，那么会根据数组的长度和其余维度的值推断出这个数。
* order ：C是横着读/写；F是竖着读/写；A是与原数组a的存储方式有关

In [2]:
# termination criteria 迭代终点标准
# _EPS
# _MAX_ITER
# 23 表示 23mm (棋盘8张不同角度照片)
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER,23,0.001)

# prepare object points, like (0,0,0),(1,0,0),(2,0,0)...(6,5,0)
#pattern_size传递内点数，内点是黑色方块相互连通的位置。为了便于辨识方向，每行每列对应的角点数不能相同 。如果行列数相同，那么函数每次画出来的角点起始位置会变化，不利于标定。 
objp = np.zeros((6*9,3),np.float32)#产生一个54行,3列的全零数组,数组类型是32位浮点数，默认行优先
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)#产生54个点坐标(x,y)赋值给前2列

# 代码分析：
objp\[:,:2\] = np.mgrid\[0:8,0:6\].T.reshape(-1,2)
1. np.mgrid生成2维矩阵(网格): x方向从0到8，步长为1；y方向从0到6，步长为1；对应48个角点,如下：

```python
[[[0 0 0 0 0 0]
  [1 1 1 1 1 1]
  [2 2 2 2 2 2]
  [3 3 3 3 3 3]
  [4 4 4 4 4 4]
  [5 5 5 5 5 5]
  [6 6 6 6 6 6]
  [7 7 7 7 7 7]]

 [[0 1 2 3 4 5]
  [0 1 2 3 4 5]
  [0 1 2 3 4 5]
  [0 1 2 3 4 5]
  [0 1 2 3 4 5]
  [0 1 2 3 4 5]
  [0 1 2 3 4 5]
  [0 1 2 3 4 5]]]

```

2. 将该矩阵转置,得：
```python
[[[0 0]
  [1 0]
  [2 0]
  [3 0]
  [4 0]
  [5 0]
  [6 0]
  [7 0]]

 [[0 1]
  [1 1]
  [2 1]
  [3 1]
  [4 1]
  [5 1]
  [6 1]
  [7 1]]

 [[0 2]
  [1 2]
  [2 2]
  [3 2]
  [4 2]
  [5 2]
  [6 2]
  [7 2]]

 [[0 3]
  [1 3]
  [2 3]
  [3 3]
  [4 3]
  [5 3]
  [6 3]
  [7 3]]

 [[0 4]
  [1 4]
  [2 4]
  [3 4]
  [4 4]
  [5 4]
  [6 4]
  [7 4]]

 [[0 5]
  [1 5]
  [2 5]
  [3 5]
  [4 5]
  [5 5]
  [6 5]
  [7 5]]]
```
3. 改变数组形状变成 只有2列 的形状
```python
 [[0 0]
 [1 0]
 [2 0]
 [3 0]
 [4 0]
 [5 0]
 [6 0]
 [7 0]
 [0 1]
 [1 1]
 [2 1]
 [3 1]
 [4 1]
 [5 1]
 [6 1]
 [7 1]
 [0 2]
 [1 2]
 [2 2]
 [3 2]
 [4 2]
 [5 2]
 [6 2]
 [7 2]
 [0 3]
 [1 3]
 [2 3]
 [3 3]
 [4 3]
 [5 3]
 [6 3]
 [7 3]
 [0 4]
 [1 4]
 [2 4]
 [3 4]
 [4 4]
 [5 4]
 [6 4]
 [7 4]
 [0 5]
 [1 5]
 [2 5]
 [3 5]
 [4 5]
 [5 5]
 [6 5]
 [7 5]]
```
4. 赋值给objp的前两列，作为x,y坐标,在一个平面上z=0，最终objp 为：
```python
[[0. 0. 0.]
 [1. 0. 0.]
 [2. 0. 0.]
 [3. 0. 0.]
 [4. 0. 0.]
 [5. 0. 0.]
 [6. 0. 0.]
 [7. 0. 0.]
 [0. 1. 0.]
 [1. 1. 0.]
 [2. 1. 0.]
 [3. 1. 0.]
 [4. 1. 0.]
 [5. 1. 0.]
 [6. 1. 0.]
 [7. 1. 0.]
 [0. 2. 0.]
 [1. 2. 0.]
 [2. 2. 0.]
 [3. 2. 0.]
 [4. 2. 0.]
 [5. 2. 0.]
 [6. 2. 0.]
 [7. 2. 0.]
 [0. 3. 0.]
 [1. 3. 0.]
 [2. 3. 0.]
 [3. 3. 0.]
 [4. 3. 0.]
 [5. 3. 0.]
 [6. 3. 0.]
 [7. 3. 0.]
 [0. 4. 0.]
 [1. 4. 0.]
 [2. 4. 0.]
 [3. 4. 0.]
 [4. 4. 0.]
 [5. 4. 0.]
 [6. 4. 0.]
 [7. 4. 0.]
 [0. 5. 0.]
 [1. 5. 0.]
 [2. 5. 0.]
 [3. 5. 0.]
 [4. 5. 0.]
 [5. 5. 0.]
 [6. 5. 0.]
 [7. 5. 0.]]
```

In [3]:
# arrays to store object points and image points from all the images.
objpoints = [] # 3D points in real world space
imgpoints = [] # 2D points in image plane.
images = glob.glob('./chessboard/*.JPG')+glob.glob('./chess/*.jpg')+glob.glob('./chess/*.png')#取图像

In [4]:
#转换成灰度图片，并查看
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    cv2.imshow('gray',gray)
    while cv2.waitKey(100) != 27: #loop if not get ESC
        if cv2.getWindowProperty('gray',cv2.WND_PROP_VISIBLE) <= 0:
            break
cv2.destroyAllWindows()

In [5]:
id_corner =0
for fname in images:
    img = cv2.imread(fname) #依次读取图片
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY) #转换成灰度图像
    ret,corners = cv2.findChessboardCorners(gray,(9,6),None)#用此函数确定是否找到了角点
    id_corner += 1
    print('corners',id_corner,':\n',corners)#像素坐标系下坐标(以像素为单位)

corners 1 :
 [[[487.44006  382.93732 ]]

 [[423.00735  384.58572 ]]

 [[364.54208  385.2679  ]]

 [[309.64523  386.4621  ]]

 [[260.8641   387.30865 ]]

 [[215.41199  388.51978 ]]

 [[172.89574  389.12714 ]]

 [[133.38406  390.30994 ]]

 [[ 97.53652  390.34045 ]]

 [[483.55515  316.2397  ]]

 [[421.59686  319.79572 ]]

 [[364.37683  323.71475 ]]

 [[310.9762   327.0295  ]]

 [[263.44373  330.48523 ]]

 [[218.54437  333.47067 ]]

 [[177.1155   336.37424 ]]

 [[137.6927   338.56058 ]]

 [[102.740715 341.39017 ]]

 [[480.31488  252.10193 ]]

 [[419.27475  259.32712 ]]

 [[364.2023   264.79968 ]]

 [[312.6792   271.49054 ]]

 [[265.71976  276.5234  ]]

 [[222.41647  281.57867 ]]

 [[181.57036  285.7733  ]]

 [[143.60298  290.54694 ]]

 [[109.20643  294.1395  ]]

 [[476.93384  191.79024 ]]

 [[417.44687  201.11328 ]]

 [[364.14102  209.5581  ]]

 [[313.45837  217.07433 ]]

 [[267.58163  224.47556 ]]

 [[225.05643  230.89325 ]]

 [[185.31104  237.69188 ]]

 [[147.80031  243.1806  ]]

 [[113.

In [6]:
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    ret,corners = cv2.findChessboardCorners(gray,(9,6),None)
    if ret == True:
        objpoints.append(objp) #物点
        corners2 = cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)#更精细的单位，亚像素
        # 导入灰度图片，导入粗略点，显示窗口大小，忽略细微结构？迭代）
        imgpoints.append(corners2)
        
        # Draw and display the corners
        img = cv2.drawChessboardCorners(img,(9,6),corners2,ret)
        cv2.imshow('img',img)
        
        while cv2.waitKey(100) != 27:
            if cv2.getWindowProperty('img',cv2.WND_PROP_VISIBLE) <= 0:
                break
cv2.destroyAllWindows()
        

In [7]:
ret,mrx,dist,rvecs,tvecs = cv2.calibrateCamera(objpoints,imgpoints,(9,6),None,None)
print(ret)
print(mrx)
print(dist)
print(rvecs)
print(tvecs)

0.330199464814
[[681.12423557   0.         303.78759146]
 [  0.         680.58815505 223.50308226]
 [  0.           0.           1.        ]]
[[ 1.52334696e-01 -1.33623071e+00 -2.30394802e-03 -1.82767380e-03
   3.61843765e+00]]
[array([[0.80525587],
       [0.47107272],
       [2.94194609]]), array([[-0.63182094],
       [ 0.4921929 ],
       [ 2.85785624]]), array([[-0.07254189],
       [-0.60534364],
       [-3.03165513]]), array([[-0.35773552],
       [-0.51996488],
       [-3.05359558]]), array([[ 0.34881064],
       [-0.82711516],
       [-2.97317339]]), array([[-0.57875824],
       [ 0.36545256],
       [ 2.69021845]]), array([[-0.39289762],
       [-0.61223739],
       [-2.63116274]]), array([[-0.70238244],
       [-0.52801675],
       [-2.9334863 ]])]
[array([[ 2.69456152],
       [ 2.35084716],
       [10.03273348]]), array([[ 4.6046759 ],
       [ 2.20742208],
       [15.3914716 ]]), array([[ 3.81779571],
       [ 2.6703888 ],
       [11.5824284 ]]), array([[ 3.46762395],
   

未矫正之前和矫正之后，长度测量，

裁剪 模糊 边缘，进一步减小误差

In [8]:
# 矫正
img = cv2.imread('./chessboard/10091602_01.jpg')
h,w = img.shape[:2] #返回图像大小
newcameramtx,roi = cv2.getOptimalNewCameraMatrix(mrx,dist,(w,h),1,(w,h))
print(h,w)
print(roi)

480 640
(7, 6, 621, 464)


roi返回值是0，可能是因为畸变系数太大了(图片边缘不清楚，我的相机有偏移，右侧被挡住一部分)，让图片四角卷起来了，可手动改相机内参矩阵的fx,fy，直到匹配

2020-10-9：换了相机之后可以写入成功了

In [9]:
dst = cv2.undistort(img,mrx,dist,None,newcameramtx)
print(dst)
#crop the image
x,y,w,h = roi
dst = dst[y:y+h,x:x+w]
cv2.imwrite('calibresult2.png',dst)

[[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]]


True